In [1]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('/content/train.csv')

In [4]:
df.head()

,labels,data
0,8,أَنا الفقير وباللَه العظيم غني # لئن فقدتك في ...
1,10,وَلوعاً بِيُمنَى نَمْنَمَتْها حَدِيقَةٌ # نَزْ...
2,11,فيا منْ لم أزلْ أحظى لديه # بفضلٍ جامعٍ بابَ ا...
3,9,وَسَلامٌ عَلَى ضَرِيحِكَ مَا أَهْ # دَتْ شَذَا...
4,8,أمِنْتُ فقري لما قُلتُ عن ثِقَةٍ # أنْ لا جواد...


In [6]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.1)

In [7]:
train.head()

,labels,data
1643,1,وتدرَجتْ فوق الخمائلِ غُدْرها # وتأرَجتْ بشذا ...
5781,13,فَعايَنَ المَوتَ الَّذي مِنهُ هَرَب # وَمَن يَ...
1098,6,وَقُلتَ لَيتَ بِكَفّي # عِنانَ جَرداءَ شَطبَه
1513,11,فَلاَ عِنْدِي لَهُ نِعَمٌ تُجَازَى # وَلا لِي ...
194,8,حُسنى دعتك الورى من جملة الرمل # كم مقلة عميت ...


In [8]:
train.labels.value_counts()

7     824
11    814
8     803
0     785
2     783
10    783
13    766
9     753
1     749
4     627
6     165
5      76
3      54
12     43
Name: labels, dtype: int64

In [9]:
indx2label = {0: 'saree', 1: 'kamel', 2: 'mutakareb', 3: 'mutadarak',
              4: 'munsareh', 5: 'madeed', 6: 'mujtath', 7: 'ramal', 8: 'baseet',
              9: 'khafeef', 10: 'taweel', 11: 'wafer', 12: 'hazaj', 13: 'rajaz'}

## Preprocess tha data

In [96]:
import re

cleaned_data = []

for line in train['data']:
  line = re.sub('\(|\)|:|«|»|#|\!|؟|؛|،|ـ|\.|,|\?', '', line)
  cleaned_data.append(line)

In [97]:
cleaned_data[:10]

['وتدرَجتْ فوق الخمائلِ غُدْرها  وتأرَجتْ بشذا العبيرِ رياحُها',
 'فَعايَنَ المَوتَ الَّذي مِنهُ هَرَب  وَمَن يَفوتُ قَدَراً إِذا اِقتَرَب',
 'وَقُلتَ لَيتَ بِكَفّي  عِنانَ جَرداءَ شَطبَه',
 'فَلاَ عِنْدِي لَهُ نِعَمٌ تُجَازَى  وَلا لِي عِنْدَهُ ذِمَمٌ تُرَاعَى',
 'حُسنى دعتك الورى من جملة الرمل  كم مقلة عميت أرجعت ناظرها',
 'عَجِبتُ مِنهُ يَشدو فَيَظهَرُ لِلأَس  ماعِ حُسنَ الإِعرابِ وَاللَحنِ',
 'يا قَمَراً عُطِّلَ الظَلامُ بِهِ  يا دُرَّةً لَم يُكِنَّها الصَدَفُ',
 'قد فاض في الأرض حيث البحر لم يفض  الحاملين من الأشياء أثقلها',
 'يُثني عَلَيهِ مِنَ الأَوقاتِ أَربَعَةٌ  يَومٌ وَشَهرٌ وَأَعوامٌ وَآبادُ',
 'هَطلاءُ تَضحَكَُ كُلُّ زَهرَةِ صَفحَةٍ  عَنها وَتُعشِبُ كُلُّ ساحَةِ دارِ']

In [98]:
letters = set(' '.join(cleaned_data))
len(letters)

45

In [99]:
chr2indx = {c:i+1 for i, c in enumerate(letters)}

In [100]:
tokenized_data = [[chr2indx[c] for c in line] for line in cleaned_data]

In [101]:
max_length = max(len(line) for line in tokenized_data)

In [122]:
max_length

95

In [102]:
from keras_preprocessing.sequence import pad_sequences

In [103]:
#padding
x = pad_sequences(tokenized_data, max_length, padding='post', value=0)

In [22]:
y = train['labels']

In [104]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.1)

In [123]:
#wrapping the preprocessing to use on test data

def preprocess_data(data, chr2indx, max_length=max_length):

  cleaned_data = []

  for line in data:
    line = re.sub('\(|\)|:|«|»|#|\!|؟|؛|،|ـ|\.|,|\?', '', line)
    cleaned_data.append(line)
  print("First 10 lines in the data:\n", cleaned_data[:10])


  tokenized_data = [[chr2indx.get(c, -1) for c in line] for line in cleaned_data]
  pad_sentences = pad_sequences(tokenized_data, max_length, padding='post', value=0)

  return pad_sentences

In [25]:
from tensorflow import keras

In [106]:
len(indx2label)

14

## Building the model

In [108]:
model = keras.Sequential([
    keras.layers.InputLayer((max_length,)),
    keras.layers.Embedding(len(chr2indx)+1, 256),
    keras.layers.Bidirectional(keras.layers.LSTM(200, return_sequences=True)),
    keras.layers.Bidirectional(keras.layers.LSTM(512, return_sequences=True)),
    keras.layers.Bidirectional(keras.layers.LSTM(512, return_sequences=True)),
    keras.layers.Bidirectional(keras.layers.LSTM(512, return_sequences=True)),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dropout(0.4),
    keras.layers.Dense(len(indx2label), activation='softmax')
])

In [109]:
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [110]:
model.summary()

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_15 (Embedding)    (None, 95, 256)           11776     
                                                                 
 bidirectional_61 (Bidirecti  (None, 95, 400)          731200    
 onal)                                                           
                                                                 
 bidirectional_62 (Bidirecti  (None, 95, 1024)         3739648   
 onal)                                                           
                                                                 
 bidirectional_63 (Bidirecti  (None, 95, 1024)         6295552   
 onal)                                                           
                                                                 
 bidirectional_64 (Bidirecti  (None, 95, 1024)         6295552   
 onal)                                               

In [111]:
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_delta=0.0001, min_lr=0.00001, verbose=1)
checkpoint = keras.callbacks.ModelCheckpoint('model_checkpoint.h5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
early_stop = keras.callbacks.EarlyStopping(patience=5)

In [112]:
model.fit(x_train, y_train, validation_data= (x_val, y_val), epochs = 50, batch_size= 128, 
          callbacks=[reduce_lr, checkpoint, early_stop])

Epoch 1/50
57/57 [==============================] - ETA: 0s - loss: 2.2919 - accuracy: 0.1896
Epoch 1: val_accuracy improved from -inf to 0.29514, saving model to model_checkpoint.h5
57/57 [==============================] - 38s 458ms/step - loss: 2.2919 - accuracy: 0.1896 - val_loss: 2.0300 - val_accuracy: 0.2951 - lr: 0.0010
Epoch 2/50
57/57 [==============================] - ETA: 0s - loss: 1.9743 - accuracy: 0.2825
Epoch 2: val_accuracy improved from 0.29514 to 0.35616, saving model to model_checkpoint.h5
57/57 [==============================] - 24s 415ms/step - loss: 1.9743 - accuracy: 0.2825 - val_loss: 1.7924 - val_accuracy: 0.3562 - lr: 0.0010
Epoch 3/50
57/57 [==============================] - ETA: 0s - loss: 1.8718 - accuracy: 0.3150
Epoch 3: val_accuracy improved from 0.35616 to 0.36239, saving model to model_checkpoint.h5
57/57 [==============================] - 24s 424ms/step - loss: 1.8718 - accuracy: 0.3150 - val_loss: 1.7564 - val_accuracy: 0.3624 - lr: 0.0010
Epoch 4/50

In [114]:
model = keras.models.load_model('/content/model_checkpoint.h5')

In [124]:
x_test = preprocess_data(test['data'], chr2indx)
y_test = test['labels']

First 10 lines in the data:
 ['لا يختشي ملاعب الظنون  والأمر مبنيٌّ على السكون', 'تُذَكِّرُني راحَةَ أَهلِ البِلى  أَرواحُ لَيلٍ بِخُزامى هَبَبنَ', 'خَليلَيَّ لَو كُنتُ الصَحيحَ وَكُنتُما  سَقيمَينِ لَم أَفعَل كَفِعلِكُما بِيا', 'هَمّي مِنَ الدُنيا خُلُوّي بِها  بِذاكَ أَدعو خالِقي في الصَلاة', 'يوم يقول لك السرور به اقترح  ما شئت من بيض الأماني يفعل', 'يَتَحَيَّرُ الراؤُونَ في نَعتَيهِما  أَصفاءُ ماءٍ أم صَفاءُ دَرارِ', 'أَضِنُّ عَن الدُنيا بِطَرفي وَطَرفِها  فَهَل بَعدَ هَذا مِن مَقالٍ لِمُشفِقِ', 'قد شَرد عن جَفْني غُمضى  حتّى ائتلفَتْ لِيَ شُرَّده', 'وَأَغَرَّ أَروَعَ مِلءِ سَمعِ المُنتَقى  حُرِّ الكَلامِ وَمِلءِ عَينِ المُبصِرِ', 'رَبَّةَ الدَّوْلَةِ وَالجَاهِ المَكِينْ  عُدْتِ يَحْدُو رَكْبَكِ الرُّوحُ الأَمِينْ']


In [126]:
model.evaluate(x_test, y_test, batch_size=128)

7/7 [==============================] - 1s 139ms/step - loss: 0.3639 - accuracy: 0.9294


[0.363949179649353, 0.9293721914291382]